In [ ]:
import numpy as np
class Softmax:
    def __init__(self):
        self.y_pred = None

    def forward(self, x, y):
        # May be overflow for larger numbers so potentially use different method
        exp_sum = np.sum(np.exp(x))
        self.y_pred = np.exp(x) / sum
        return self.y_pred

    # Using Cross-Entropy-Loss instead so no backward pass here
    def backward(self, y_actual):
        return self.y_pred - y_actual

Input layer: 3072 nodes<br>
Output classification layer: 10 nodes

In [ ]:
import math
import numpy as np
class NeuralNetwork:
    def __init__(self, x_train, y_train, hidden_layers, neurons, activation_func, learning_rate=0.03):
        self.inputs = x_train
        self.outputs = y_train

        self.hidden_layers = hidden_layers

        if (len(neurons) != hidden_layers):
            raise ValueError("Neurons array length mismatch with hidden layers amount")
        self.neurons = neurons

        if not isinstance(activation_func, (Sigmoid, ReLU, LeakyReLU)):
            raise TypeError("Activation function must be of type Sigmoid, ReLU or LeakyReLU")
        self.activation_func = activation_func

        self.learning_rate = learning_rate

        self.weights = self.create_weight_matrices()

    def create_weight_matrices(self):
        network_layout = [self.inputs]
        for i in range(len(self.hidden_layers)):
            network_layout.append(self.neurons[i])
        network_layout.append(self.outputs)
        
        weights = []

        # Specific weight initialisation for whatever activation function is used

        # Xavier weight initialisation (Sigmoid)
        if isinstance(self.activation_func, Sigmoid):
            for i in range(len(l) -  1):
                input = l[i]
                output = l[i+1]
                weight_init = (np.random.uniform(low=-1, high=1, size=(output, input))) / (np.sqrt(input))
                weights.append(weight_init)
        
        # He wight initialisation (ReLU)
        if isinstance(self.activation_func, (ReLU, LeakyReLU)):
            for i in range(len(l) -  1):
                input = l[i]
                output = l[i+1]
                weight_init = (np.random.uniform(low=-1, high=1, size=(output, input))) / (np.sqrt(input) / 2)
                weights.append(weight_init)

        return weights
    
    def train(self, epochs=100):
        # Inputs -> Hidden Layers -> Ouputs
        
        # Forward pass
        for i in range(epochs):
            res_at_each_layer = []
            for j in range(len(self.weights)):
                z1 = np.dot(self.inputs, self.weights[i].T)
                a1 = self.activation_func.forward(z1)
                res_at_each_layer.append(z1)
                res_at_each_layer.append(a1)
            # Apply softmax
            softmax = Softmax()
            softmax.forward = res_at_each_layer[len(res_at_each_layer) - 1]
            res_at_each_layer.append(softmax.y_pred)
            # Compute CLE_loss and use to get actual loss then backprop through 
            pass
        pass 

    def run(self):
        pass       